# Actor-Critic

In [1]:
import tensorflow as tf
import numpy as np
import gym
import tqdm
import itertools
import statistics

from matplotlib import pyplot as plt
from typing import List, Tuple
from tensorflow import keras
from tensorflow.keras import layers

print(tf.__version__)
print(gym.__version__)

logdir = 'logs/actor-critic'
file_writer = tf.summary.create_file_writer(logdir + "/metrics")
file_writer.set_as_default()
%tensorboard --logdir logs/actor-critic


c:\Users\37103\AppData\Local\Programs\Python\Python39\lib\site-packages\gym\envs\registration.py:216: UserWarning: WARN: Overriding environment ALE/Tetris-v5
  logger.warn("Overriding environment {}".format(id))
c:\Users\37103\AppData\Local\Programs\Python\Python39\lib\site-packages\gym\envs\registration.py:216: UserWarning: WARN: Overriding environment ALE/Tetris-ram-v5
  logger.warn("Overriding environment {}".format(id))
c:\Users\37103\AppData\Local\Programs\Python\Python39\lib\site-packages\gym\envs\registration.py:216: UserWarning: WARN: Overriding environment ALE/Adventure-v5
  logger.warn("Overriding environment {}".format(id))
c:\Users\37103\AppData\Local\Programs\Python\Python39\lib\site-packages\gym\envs\registration.py:216: UserWarning: WARN: Overriding environment ALE/Adventure-ram-v5
  logger.warn("Overriding environment {}".format(id))
c:\Users\37103\AppData\Local\Programs\Python\Python39\lib\site-packages\gym\envs\registration.py:216: UserWarning: WARN: Overriding enviro

2.8.0
0.21.0


UsageError: Line magic function `%tensorboard` not found.


In [2]:
#* Hyper-parameters
env = gym.make('CartPole-v1')
num_states = env.observation_space.shape[0]
num_actions = env.action_space.n  # 2
num_hidden_units = 128

max_episodes = 10000
max_step_per_episode = 1000
min_episodes_length = 100
# Cartpole-v0 is considered solved if average reward is >= 195 over 100 consecutive trials
running_reward = 0
reward_threshold = 195

gamma = 0.99 # Discount rate

In [3]:
# class ActorCritic(tf.keras.Model):
#   """Combined actor-critic network.
  
#     It returns an actor network and a critic network.
#   """

#   def __init__(
#           self,
#           num_actions: int,
#           num_hidden_units: int):
#     """Initialize."""
#     super().__init__()
#     # self.inputs = keras.Input(shape=(num_states,), name='state')
#     self.common = layers.Dense(num_hidden_units, activation="relu", name="common")
#     self.actor_layer = layers.Dense(num_actions, name="actor") # Policy
#     self.critic = layers.Dense(1, name='critic')
    
#     self.optimizer = tf.keras.optimizers.Adam(learning_rate=0.01)

#   # 前向传播
#   def call(self, inputs):
#     '''
#     Args:
#       inputs: Observation tensor
#     Returns:
#       actor_logits: Logits for policy network
#       critic_value: Value of the current state
#     '''
#     x = self.common(inputs)
#     self.actor = self.actor_layer(x)
#     return self.actor_layer(x), self.critic(x)
  
#   def choose_action(self):
#     '''Choose an action according to the policy network.
    
#     Args:
#       state: Current state
#     Returns:
#       action: Action to take
#     '''
#     actor_logits = self.actor
#     # Sample action from policy distribution
#     action = tf.random.categorical(actor_logits, 1)[0, 0]
#     return action.numpy()
  
# #todo 还是用函数式API更好 明天的任务

In [4]:
# Define Functional model
inputs_layer = keras.Input(shape=(num_states))
common_layer = layers.Dense(128, activation="relu", name="common")(inputs_layer)
actor_layer = layers.Dense(env.action_space.n, name="actor")(common_layer)
critic_layer = layers.Dense(1, name='critic')(common_layer)

# actor_network = keras.Model(inputs=inputs_layer, outputs=actor_layer)
# critic_network = keras.Model(inputs=inputs_layer, outputs=critic_layer)
# target_critic_network = keras.Model(inputs=inputs_layer, outputs=critic_layer)

In [5]:
# model = keras.Model(inputs=inputs_layer, outputs=[actor_layer, critic_layer])

# input = np.random.random((1, 1, num_states))
# model(input)
#print(test.summary())

In [6]:
# print(actor_network.summary())

In [7]:
class Actor():
    
    def __init__(self) -> None:
        self.actor_network = keras.Model(inputs=inputs_layer, outputs=actor_layer)
        self.optimizer = tf.keras.optimizers.Adam(learning_rate=0.01)
    
    def choose_action(self, state):
        '''根据state选择action'''
        actor_logits = self.actor_network(state)
        # Sample action from policy distribution
        # print('actor logits: {}'.format(actor_logits))
        action = tf.random.categorical(actor_logits, 1)[0, 0]
        return action.numpy()
    
    def learn(self, state, action ,TD_error):
        with tf.GradientTape() as tape:
            actor_logits = self.actor_network(state) # [x1, x2]
            # Calculate the loss
            #* （1）动作概率分布向真实动作的概率分布靠拢 （2）TD-error为正，说明当前动作是正确的，反之，说明当前动作是错误的
            cross_entropy = tf.nn.sparse_softmax_cross_entropy_with_logits(
                labels=action, logits=actor_logits)
            # loss为两者的乘积
            loss = tf.reduce_mean(cross_entropy * TD_error)
        grad = tape.gradient(loss, self.actor_network.trainable_variables)
        self.optimizer.apply_gradients(zip(grad, self.actor_network.trainable_variables))

In [8]:
# input = np.random.random((1, num_states))
# print(actor_critic(input))
# print(actor_critic(input)[0])
# print(actor_critic(input)[1]) 
# # print([input])
# actor = Actor()
# data = np.random.randint(2, size=(1, 10, 4))
# output = actor.choose_action(data)
# print(input)
# print(output)


In [9]:
state = env.reset()
print(state.shape)

(4,)


In [10]:
class Critic():

    def __init__(self) -> None:
        self.critic_network = keras.Model(
            inputs=inputs_layer, outputs=critic_layer)
        self.optimizer = tf.keras.optimizers.Adam(learning_rate=0.01)
    
    def learn(self, state, reward ,next_state):
        '''更新价值网络，并计算TD error'''
        with tf.GradientTape() as tape:
            # Calculate critic loss
            state_value = self.critic_network(state)
            next_state_value = self.critic_network(next_state)
            TD_error = reward + gamma * next_state_value - state_value
            loss = tf.math.square(TD_error)
        # Calculate gradients using critic loss
        grad = tape.gradient(loss, self.critic_network.trainable_variables)
        # Apply critic gradients
        self.optimizer.apply_gradients(zip(grad, self.critic_network.trainable_variables))
        return TD_error

In [11]:
# class Actor():
    
#     def __init__(self):
#         super().__init__()
#         inputs = keras.Input(shape=(num_states,))
#         common = layers.Dense(128, activation="relu", name="common")(inputs)
#         actor = layers.Dense(env.action_space.n, name="actor")(common)
#         self.model = keras.Model(inputs=inputs, outputs=actor)
#         # self.model = keras.Model(inputs=inputs, outputs=actor)
        
#     def call(self, inputs):
#         return self.model(inputs)

In [12]:
num_actions = env.action_space.n  # 2
num_hidden_units = 128

# actor_critic = ActorCritic(num_actions, num_hidden_units)

# test = tf.ones((1, 4))
# actor_critic(test)



Train

In [13]:


actor = Actor()
critic = Critic()

for i in range(max_episodes):
    state = np.expand_dims(env.reset(), axis=0)
    running_reward = 0
    # state = env.reset()
    for j in itertools.count(): # 因为是连续状态,所以无法设置一个固定的最大episode长度
        # 获得当前动作
        action = actor.choose_action(state)
        # 获得下一个状态和奖励
        next_state, reward, done, _ = env.step(action)
        next_state = np.expand_dims(next_state, axis=0)
        # print('step {} | next state: {}'.format(j+1, next_state))
        # 获得下一动作
        next_action = actor.choose_action(next_state)
        
        # episodes_reward.append(episode_reward)
        # running_reward = statistics.mean(episodes_reward)
        running_reward += reward
        # 更新价值网络
        TD_error = critic.learn(state, reward, next_state)
        
        actor.learn(state, [action], TD_error)
        
        state = next_state
        
        if done:
            tf.summary.scalar('running_reward / episode', running_reward, step=i)
            # print('Episode {} finished after {} timesteps, reward: {}'.format(i+1, j+1, running_reward))
            break
        
    if running_reward >= 475:
        break
    # if running_reward > reward_threshold and j >= min_episodes_length:
    #     break

actor logits: [[0.00498368 0.0038194 ]]
actor logits: [[-0.01511516  0.04919631]]
actor logits: [[-0.04218998  0.09518117]]
actor logits: [[-0.06954426  0.14710352]]
actor logits: [[-0.14918692  0.26603445]]
actor logits: [[-0.102944   0.1855932]]
actor logits: [[-0.08520375  0.16742805]]
actor logits: [[-0.0803791  0.1268283]]
actor logits: [[-0.07778304  0.12110601]]
actor logits: [[-0.0627915   0.13237144]]
actor logits: [[-0.05461703  0.11604031]]
actor logits: [[-0.08424582  0.12749088]]
actor logits: [[-0.08006249  0.12128927]]
actor logits: [[-0.03795867  0.08907079]]
actor logits: [[-0.02985122  0.07603086]]
actor logits: [[-0.08295958  0.12423927]]
actor logits: [[-0.07670823  0.1165531 ]]
actor logits: [[-0.00939136  0.04866938]]
actor logits: [[0.00224512 0.03645913]]
actor logits: [[ 0.0651869  -0.02726635]]
actor logits: [[ 0.06980858 -0.03967924]]
actor logits: [[ 0.13714434 -0.1090248 ]]
actor logits: [[ 0.12218384 -0.09469095]]
actor logits: [[ 0.05194658 -0.03284175]]


KeyboardInterrupt: 

---